## Импорт библиотек

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import random_split, Dataset, DataLoader
from torchvision import transforms, datasets
import torch.optim as optim

from tqdm.auto import tqdm

import time
from pathlib import Path
import os
from PIL import Image
import pandas as pd
import numpy as np

import glob
import pickle

In [2]:
# проверяем доступность тренировки на GPU
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [3]:
# Задаем долю валидационной выборки
SIZE_VALID_DATA: float = 0.1
# Размер батчей
BATCH_SIZE: int = 64
TEST_DIR = Path('../data/journey-springfield/testset')
RESCALE_SIZE = 224

## Загрузка и подготовка данных

In [4]:
train_transform = transforms.Compose([
        transforms.RandomResizedCrop(RESCALE_SIZE), # обрезаем случайную часть изображения до заданного размера
        transforms.RandomHorizontalFlip(p = 0.2),
        # transforms.RandomVerticalFlip(p = 0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

In [5]:
class ImageDataset(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        self.len_ = len(self.files)

    def __len__(self):
        return self.len_

    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image

    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        x = self.load_sample(self.files[index])
        x = self._prepare_sample(x)
        x = np.array(x / 255, dtype='float32')
        x = transform(x)
        return x

    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)

In [6]:
train_dataset = datasets.ImageFolder(
    root='../data/journey-springfield/train/simpsons_dataset/',
    transform=data_transform
                                    )
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))
test_dataset = ImageDataset(test_files)

In [7]:
n_classes = len(train_dataset.classes)

In [8]:
size_dataset = len(train_dataset)
print(f"Размер обучающей выборки:", size_dataset)

Размер обучающей выборки: 20933


In [9]:
train, valid = random_split(train_dataset, [int(size_dataset * (1 - SIZE_VALID_DATA)),
                                            size_dataset - int(size_dataset * (1 - SIZE_VALID_DATA))])

In [10]:
print(f"train:{len(train)}", f"test:{len(valid)}",f"sum: {len(train) + len(valid)}", sep="\n")

train:18839
test:2094
sum: 20933


In [11]:
train_loader = DataLoader(
    train, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=8, pin_memory=True
                        )

valid_loader = DataLoader(
    valid, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=8, pin_memory=True
)

## Создаем модель нейронной сети

In [12]:
class SimpleModel(nn.Module):

    def __init__(self, n_classes):
        super().__init__()
        self.dropout = nn.Dropout(0.2)
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(8)
            
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(16)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(32)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(64)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.BatchNorm2d(128)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(128 * 5 * 5, 512),
            nn.ReLU()
        )
        self.out = nn.Linear(512, n_classes)
        self.dropout = nn.Dropout(0.25)
        

    def forward(self, x):
        x = self.dropout(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        # x = x.view(x.size(0), -1)
        logits = self.out(x)
        return logits

## Функции обучение и валидации

In [13]:
def train(model, trainloader, optimizer, criterion):
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        # calculate the loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()
    
    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc


# validation
def validate(model, testloader, criterion):
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader)):
            counter += 1
            
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            # forward pass
            outputs = model(image)
            # calculate the loss
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            # calculate the accuracy
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
        
    # loss and accuracy for the complete epoch
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    return epoch_loss, epoch_acc

## Обучение

In [14]:
device = torch.device("cuda")
model = SimpleModel(n_classes=n_classes).to(device)
# learning_parameters 
lr = 1e-3
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=0.01)
# loss function
criterion = nn.CrossEntropyLoss()
epochs: int = 30
# lists to keep track of losses and accuracies
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []
# start the training
for epoch in range(epochs):
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_acc = train(model, train_loader, 
                                              optimizer, criterion)
    valid_epoch_loss, valid_epoch_acc = validate(model, valid_loader,  
                                                 criterion)
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    print('-'*50)
    time.sleep(1)
    
# save the trained model weights
# save_model(epochs, model, optimizer, criterion)
# save the loss and accuracy plots
# save_plots(train_acc, valid_acc, train_loss, valid_loss)
print('TRAINING COMPLETE')

[INFO]: Epoch 1 of 30
Training


  0%|          | 0/295 [00:00<?, ?it/s]

Validation


  0%|          | 0/33 [00:00<?, ?it/s]

Training loss: 2.474, training acc: 32.990
Validation loss: 2.562, validation acc: 29.943
--------------------------------------------------
[INFO]: Epoch 2 of 30
Training


...

Training loss: 1.357, training acc: 64.324
Validation loss: 1.459, validation acc: 63.754
--------------------------------------------------
[INFO]: Epoch 30 of 30
Training


  0%|          | 0/295 [00:00<?, ?it/s]

Validation


  0%|          | 0/33 [00:00<?, ?it/s]

Training loss: 1.348, training acc: 64.616
Validation loss: 1.430, validation acc: 63.181
--------------------------------------------------
TRAINING COMPLETE


## Предсказание

In [15]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []

        for inputs in test_loader:
            inputs = inputs.to(device)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)

    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [16]:
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)
probs = predict(model, test_loader)
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))
preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))
test_filenames = [path.name for path in test_dataset.files]

In [17]:
my_submit = pd.read_csv("../data/journey-springfield/sample_submission.csv")
my_submit = pd.DataFrame({'Image_id': test_filenames, 'Expected': preds})
my_submit = my_submit.rename(columns={"Image_id": "Id"})
my_submit.head()

,Id,Expected
0,img0.jpg,nelson_muntz
1,img1.jpg,nelson_muntz
2,img10.jpg,ned_flanders
3,img100.jpg,chief_wiggum
4,img101.jpg,apu_nahasapeemapetilon


In [18]:
my_submit.to_csv('simple_cnn_batchorm.csv', index=False)